<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

#  K-Nearest Neighbors

_Authors: Alex Sherman (DC)_

In this lesson, we will get an intuitive and practical feel for the **k-Nearest Neighbors** model. 

- kNN is a **non-parametric model**, meaning it is non-linear. So, the model is not represented as an equation with parameters (e.g. the $\beta$ values in linear regression).
<br>

<a id="k-nearest-neighbors-knn-classification"></a>

## K-Nearest Neighbors (KNN) Classification
---

K-nearest neighbors classification is (as its name implies) a classification model that uses the "K" most similar observations in order to make a prediction.

KNN is a supervised learning method; therefore, the training data must have known target values.

The process of of prediction using KNN is fairly straightforward:

1. Pick a value for K.
2. Search for the K observations in the data that are "nearest" to the measurements of the unknown iris.
    - Euclidian distance is often used as the distance metric, but other metrics are allowed.
3. Use the most popular response value from the K "nearest neighbors" as the predicted response value for the unknown Target Class.

In [1]:
#Import Libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [2]:
#import "Classified Data", set the index_col to 0
df = pd.read_csv("data/Classified Data",index_col=0)
df.describe()

,WTT,PTI,EQW,SBI,LQE,QWG,FDJ,PJF,HQE,NXJ,TARGET CLASS
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.00000
mean,0.949682,1.114303,0.834127,0.682099,1.032336,0.943534,0.963422,1.071960,1.158251,1.362725,0.50000
std,0.289635,0.257085,0.291554,0.229645,0.243413,0.256121,0.255118,0.288982,0.293738,0.204225,0.50025
min,0.174412,0.441398,0.170924,0.045027,0.315307,0.262389,0.295228,0.299476,0.365157,0.639693,0.00000
25%,0.742358,0.942071,0.615451,0.515010,0.870855,0.761064,0.784407,0.866306,0.934340,1.222623,0.00000
50%,0.940475,1.118486,0.813264,0.676835,1.035824,0.941502,0.945333,1.065500,1.165556,1.375368,0.50000
75%,1.163295,1.307904,1.028340,0.834317,1.198270,1.123060,1.134852,1.283156,1.383173,1.504832,1.00000
max,1.721779,1.833757,1.722725,1.634884,1.650050,1.666902,1.713342,1.785420,1.885690,1.893950,1.00000


<a id="standardizing-features"></a>
## Standardizing Features
---

There is one major issue that applies to many machine learning models: They are sensitive to feature scale. 

> KNN in particular is sensitive to feature scale because it (by default) uses the Euclidean distance metric. To determine closeness, Euclidean distance sums the square difference along each axis. So, if one axis has large differences and another has small differences, the former axis will contribute much more to the distance than the latter axis.

This means that it matters whether our feature are centered around zero and have similar variance to each other.

In [ ]:
#We use a scaler from sklearn

##Import
from sklearn.preprocessing import StandardScaler

##Instantiate
scaler = StandardScaler()

#fit + transform
scaled_features = scaler.fit_transform(df.drop("TARGET CLASS", axis = 1))

In [ ]:
#append the new column to the df
df_feat = pd.DataFrame(scaled_features, columns = df.columns[:-1])
df_feat.describe()

## Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(scaled_features, df["TARGET CLASS"],
                                                   test_size = .3, random_state = 42)

## Using kNN
Remember that we are trying to come up with a model to predict whether someone will TARGET CLASS or not. We'll start with k = 1.

In [ ]:
#Import 
from sklearn.neighbors import KNeighborsClassifier

#Instantiate
knn = KNeighborsClassifier(n_neighbors = 1)

#Fit
knn.fit(X_train, y_train)

#predict
pred = knn.predict(X_test)

In [ ]:
#Evaluate
from sklearn.metrics import classification_report, confusion_matrix
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

## Choosing a K value
How do we make sure we can choose a good K value?

In [ ]:
error_rate = []

# Will take some time
for i in range(1,40):
    
    knn = KNeighborsClassifier(n_neighbors=i)
    knn.fit(X_train,y_train)
    pred_i = knn.predict(X_test)
    error_rate.append(np.mean(pred_i != y_test))

In [ ]:
plt.figure(figsize=(10,6))
plt.plot(range(1,40),error_rate,color='blue', linestyle='dashed', marker='o',
         markerfacecolor='red', markersize=10)
plt.title('Error Rate vs. K Value')
plt.xlabel('K')
plt.ylabel('Error Rate')

In [ ]:
#... seems like a lot of work... why not...
#from sklearn import grid_search


from sklearn.model_selection import GridSearchCV

k = np.arange(100) + 1
knn = KNeighborsClassifier()

params = {'n_neighbors': k}

#Note! CV should be no more than 25% of the data!
gs = GridSearchCV(    
    estimator= knn,
    param_grid= params,
    cv=5)

gs.fit(X_train, y_train)

#Check the documentation! How do you get only the best k? 

In [ ]:
#Now with K = ?
knn = KNeighborsClassifier(n_neighbors = 5)

knn.fit(X_train,y_train)
pred = knn.predict(X_test)

tn, fp, fn, tp =confusion_matrix(y_test,pred).ravel()

print('WITH K=5')
print('\n')
print(confusion_matrix(y_test,pred))
print('\n')
print('Confusion Matrix Expanded')
print('True Negative:{}'.format(tn), 'False Positive:{}'.format(fp))
print('False Negative:{}'.format(fn), 'True Positive:{}'.format(tp))
print('\n')
print(classification_report(y_test,pred))